In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2020125/use this test.csv
/kaggle/input/2020125/use_this.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
import os
from sklearn.preprocessing import StandardScaler


In [3]:
# get preprocessed data

total_train = pd.read_csv('../input/2020125/use_this.csv')

total_train = total_train.reset_index()

total_train.head()
print(total_train.shape)


(80176, 135)


In [4]:
total_train.head()

,index,Index,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
0,0,0,1.0,0,0,174,0.124821,0.389041,1.857143,1.580645,...,0,0,1,0,0,0,0,0,0,2
1,1,1,0.0,0,0,4,0.022599,0.010959,0.428571,0.096774,...,0,0,1,0,0,0,0,0,0,1
2,2,2,0.0,0,0,4275,11.585366,11.575342,13.285714,3.366667,...,1,0,0,1,0,0,0,0,0,1
3,3,3,2.0,0,0,280,0.329800,0.526027,2.857143,1.366667,...,0,0,0,0,1,0,0,0,0,1
4,4,4,0.0,1,0,98,0.053552,0.180822,3.000000,0.870968,...,0,0,1,0,0,0,0,0,0,1


In [5]:
# delete things useless
total_train = total_train.iloc[:,2:135]

In [6]:
total_train.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
0,1.0,0,0,174,0.124821,0.389041,1.857143,1.580645,0.766304,35,...,0,0,1,0,0,0,0,0,0,2
1,0.0,0,0,4,0.022599,0.010959,0.428571,0.096774,0.021739,1,...,0,0,1,0,0,0,0,0,0,1
2,0.0,0,0,4275,11.585366,11.575342,13.285714,3.366667,6.329670,4,...,1,0,0,1,0,0,0,0,0,1
3,2.0,0,0,280,0.329800,0.526027,2.857143,1.366667,0.530055,0,...,0,0,0,0,1,0,0,0,0,1
4,0.0,1,0,98,0.053552,0.180822,3.000000,0.870968,0.347826,2,...,0,0,1,0,0,0,0,0,0,1


In [7]:
total_train.describe()


,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
count,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,...,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000
mean,4.848655,0.472710,0.002856,2868.760190,5.914165,4.794849,10.004608,6.959703,5.536113,7.792008,...,0.162256,0.000012,0.727001,0.128368,0.077018,0.033838,0.024845,0.000162,0.000324,1.550327
std,6.264554,3.178542,0.053368,4876.761462,8.683963,7.598994,18.933466,9.981666,8.278283,305.238305,...,0.368687,0.003532,0.445503,0.334501,0.266622,0.180813,0.155655,0.012733,0.018005,1.717000
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,58.000000,0.167162,0.106849,0.571429,0.290323,0.147541,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.000000,0.000000,0.000000,465.000000,0.808812,0.734247,1.857143,1.354839,0.912568,2.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,5.500000,0.000000,0.000000,2811.000000,14.098305,5.589041,13.857143,11.900000,7.760870,8.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,21.000000,179.000000,1.000000,19513.000000,46.625000,24.526027,159.714286,40.580645,27.371585,83481.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [8]:
list(total_train.describe()['ccs'])[1]

0.4727100379165835

In [9]:
# removing outliers based on the threshold
# too large data and too little data will be replaced with mean

def get_threshold(a):
    b = 0.98
    k = []
    m = 0
    if a in list(total_train.columns):
        order = list(total_train.columns).index(a)
        a = total_train.iloc[:,order].value_counts()
        a = pd.DataFrame(a).reset_index()
        
        for i in range(len(a)):
            if m < (len(total_train) * b):
                m += a.iloc[i,1]
            else:
                k.append(a.iloc[i,0])
    else:
        print('not in the columns!')
    return k

In [11]:
# columns that ourliers need to be removed 

lines_needed = ['hour_offset', 'ccs', 'total_mails_by_sender', 'sender_freq_total_period', 'sender_freq_prev_year',
     'sender_freq_prev_week', 'sender_freq_prev_month', 'sender_freq_prev_six_months', 'images', 'urls',
    'chars_in_subject','chars_in_body', 'image_mean_value', 'image_std_value', 'urls_mean_value', 'urls_std_value',
     'chars_in_subject_mean_value', 'chars_in_subject_std_value', 'chars_in_body_mean_value', 'chars_in_body_std_value',
     'img_url_ratio', 'body_subj_ratio']
description = total_train.describe()
for item in lines_needed:
    values_need_to_be_deleted = get_threshold(item)
    for i in range(len(total_train)):
        if (total_train.at[i, item] in values_need_to_be_deleted):
            total_train.at[i, item] = list(description[item])[1]
    print('finished the ' + str(lines_needed.index(item)))

total_train.describe()

finished the 0
finished the 1
finished the 2
finished the 3
finished the 4
finished the 5
finished the 6
finished the 7
finished the 8
finished the 9
finished the 10
finished the 11
finished the 12
finished the 13
finished the 14
finished the 15
finished the 16
finished the 17
finished the 18
finished the 19
finished the 20
finished the 21


,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
count,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,...,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000,80176.000000
mean,4.826773,0.290561,0.002856,2779.418966,6.017635,4.737914,9.526241,6.872992,5.529487,5.580797,...,0.162256,0.000012,0.727001,0.128368,0.077018,0.033838,0.024845,0.000162,0.000324,1.550327
std,6.257067,0.827943,0.053368,4715.188549,8.647902,7.539546,18.532830,9.894086,8.237693,8.051847,...,0.368687,0.003532,0.445503,0.334501,0.266622,0.180813,0.155655,0.012733,0.018005,1.717000
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,58.000000,0.178571,0.106849,0.571429,0.290323,0.147541,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.000000,0.000000,0.000000,465.000000,0.881269,0.734247,1.857143,1.357143,0.912568,2.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,5.500000,0.000000,0.000000,2868.000000,14.086957,5.054795,11.714286,10.366667,7.201087,8.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,20.000000,5.000000,1.000000,19496.000000,46.625000,24.526027,159.714286,40.580645,27.371585,38.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [12]:
total_train.to_csv('total_train_removed_outlier_with_0.98.csv')

In [13]:
# standardize the columns
stdsc=StandardScaler()
normalized_total_train = stdsc.fit_transform(total_train)
normalized_total_train = pd.DataFrame(normalized_total_train)

In [14]:
total_train.head()
for_special_purpose = total_train

In [15]:
normalized_total_train.head()

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
0,-0.611596,-0.350945,-0.05352,-0.552562,-0.681420,-0.576812,-0.413814,-0.534903,-0.578222,3.653744,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,0.261896
1,-0.771416,-0.350945,-0.05352,-0.588616,-0.693240,-0.626959,-0.490898,-0.684880,-0.668607,-0.568916,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
2,-0.771416,-0.350945,-0.05352,0.317186,0.643828,0.906881,0.202856,-0.354388,0.097137,-0.196329,...,2.272250,-0.003532,-1.631874,2.605787,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
3,-0.451776,-0.350945,-0.05352,-0.530082,-0.657717,-0.558643,-0.359855,-0.556530,-0.606901,-0.693112,...,-0.440093,-0.003532,-1.631874,-0.383761,3.461787,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
4,-0.771416,0.856875,-0.05352,-0.568680,-0.689661,-0.604429,-0.352147,-0.606631,-0.629022,-0.444720,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519


In [16]:
# change the names because they are all numbers now
rename_dict = {}
for i in range(133):
    rename_dict.update({i: column_name[i]})

In [17]:
rename_dict

{0: 'hour_offset',
 1: 'ccs',
 2: 'bcced',
 3: 'total_mails_by_sender',
 4: 'sender_freq_total_period',
 5: 'sender_freq_prev_year',
 6: 'sender_freq_prev_week',
 7: 'sender_freq_prev_month',
 8: 'sender_freq_prev_six_months',
 9: 'images',
 10: 'urls',
 11: 'salutations',
 12: 'designation',
 13: 'chars_in_subject',
 14: 'chars_in_body',
 15: 'image_mean_value',
 16: 'image_std_value',
 17: 'urls_mean_value',
 18: 'urls_std_value',
 19: 'chars_in_subject_mean_value',
 20: 'chars_in_subject_std_value',
 21: 'chars_in_body_mean_value',
 22: 'chars_in_body_std_value',
 23: 'img_url_ratio',
 24: 'body_subj_ratio',
 25: 'has_subject',
 26: 'has_body',
 27: 'MailType_Class_0',
 28: 'MailType_Class_multipart/IDM',
 29: 'MailType_Class_multipart/alternative',
 30: 'MailType_Class_multipart/mixed',
 31: 'MailType_Class_multipart/related',
 32: 'MailType_Class_multipart/report',
 33: 'MailType_Class_multipart/signed',
 34: 'MailType_Class_nan',
 35: 'MailType_Class_text/calendar',
 36: 'MailTyp

In [18]:
# rename them
normalized_total_train.rename(columns = rename_dict, inplace = True)

In [19]:
normalized_total_train.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
0,-0.611596,-0.350945,-0.05352,-0.552562,-0.681420,-0.576812,-0.413814,-0.534903,-0.578222,3.653744,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,0.261896
1,-0.771416,-0.350945,-0.05352,-0.588616,-0.693240,-0.626959,-0.490898,-0.684880,-0.668607,-0.568916,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
2,-0.771416,-0.350945,-0.05352,0.317186,0.643828,0.906881,0.202856,-0.354388,0.097137,-0.196329,...,2.272250,-0.003532,-1.631874,2.605787,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
3,-0.451776,-0.350945,-0.05352,-0.530082,-0.657717,-0.558643,-0.359855,-0.556530,-0.606901,-0.693112,...,-0.440093,-0.003532,-1.631874,-0.383761,3.461787,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519
4,-0.771416,0.856875,-0.05352,-0.568680,-0.689661,-0.604429,-0.352147,-0.606631,-0.629022,-0.444720,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,-0.320519


In [20]:
# replace the number in label with the correct integers
for i in range(len(normalized_total_train)):
    normalized_total_train.iloc[i,132] = total_train.iloc[i,132]

In [21]:
normalized_total_train.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
0,-0.611596,-0.350945,-0.05352,-0.552562,-0.681420,-0.576812,-0.413814,-0.534903,-0.578222,3.653744,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,2.0
1,-0.771416,-0.350945,-0.05352,-0.588616,-0.693240,-0.626959,-0.490898,-0.684880,-0.668607,-0.568916,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1.0
2,-0.771416,-0.350945,-0.05352,0.317186,0.643828,0.906881,0.202856,-0.354388,0.097137,-0.196329,...,2.272250,-0.003532,-1.631874,2.605787,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1.0
3,-0.451776,-0.350945,-0.05352,-0.530082,-0.657717,-0.558643,-0.359855,-0.556530,-0.606901,-0.693112,...,-0.440093,-0.003532,-1.631874,-0.383761,3.461787,-0.187145,-0.159619,-0.012735,-0.018011,1.0
4,-0.771416,0.856875,-0.05352,-0.568680,-0.689661,-0.604429,-0.352147,-0.606631,-0.629022,-0.444720,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1.0


In [22]:
# make the floats into integers
normalized_total_train['label'] = normalized_total_train['label'].astype(int)

In [23]:
normalized_total_train.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed,label
0,-0.611596,-0.350945,-0.05352,-0.552562,-0.681420,-0.576812,-0.413814,-0.534903,-0.578222,3.653744,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,2
1,-0.771416,-0.350945,-0.05352,-0.588616,-0.693240,-0.626959,-0.490898,-0.684880,-0.668607,-0.568916,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1
2,-0.771416,-0.350945,-0.05352,0.317186,0.643828,0.906881,0.202856,-0.354388,0.097137,-0.196329,...,2.272250,-0.003532,-1.631874,2.605787,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1
3,-0.451776,-0.350945,-0.05352,-0.530082,-0.657717,-0.558643,-0.359855,-0.556530,-0.606901,-0.693112,...,-0.440093,-0.003532,-1.631874,-0.383761,3.461787,-0.187145,-0.159619,-0.012735,-0.018011,1
4,-0.771416,0.856875,-0.05352,-0.568680,-0.689661,-0.604429,-0.352147,-0.606631,-0.629022,-0.444720,...,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011,1


In [24]:
# split the data
X_train, X_test, Y_train, Y_test = train_test_split(normalized_total_train.iloc[:, :132], normalized_total_train['label'], train_size = 0.8)

In [57]:
Y_test.shape

(16036,)

In [25]:
X_train.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_multipart,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed
56439,-0.771416,-0.350945,-0.05352,-0.542806,-0.596866,-0.548468,-0.444648,-0.583809,-0.557110,-0.693112,...,0.453591,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011
3729,-0.771416,-0.350945,-0.05352,-0.517357,-0.651925,-0.552102,-0.421522,-0.629454,-0.578881,-0.693112,...,-2.204629,2.272250,-0.003532,-1.631874,-0.383761,-0.288868,5.343456,-0.159619,-0.012735,-0.018011
46037,0.107595,-0.350945,-0.05352,0.762770,1.133658,1.688155,0.025563,0.814668,1.828002,-0.693112,...,0.453591,-0.440093,-0.003532,0.612793,-0.383761,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011
12933,-0.771416,-0.350945,-0.05352,-0.560833,-0.655928,-0.583353,-0.460064,-0.611638,-0.639724,2.535981,...,-2.204629,2.272250,-0.003532,-1.631874,2.605787,-0.288868,-0.187145,-0.159619,-0.012735,-0.018011
37590,0.107595,-0.350945,-0.05352,2.772458,1.445569,1.904004,0.773276,0.808364,1.126189,-0.693112,...,0.453591,-0.440093,-0.003532,-1.631874,-0.383761,3.461787,-0.187145,-0.159619,-0.012735,-0.018011


In [26]:
#Running grid search for DecisionTree
import sklearn.tree as tree
from sklearn.model_selection import GridSearchCV
import sklearn.ensemble as ensemble

In [27]:
# use GridSearchCV to find the best one
# the param_grid only contains very few parameters because for convenience I just post the final best ones...
param_grid = {'criterion': ['entropy', 'gini'],
              'max_depth': [10],
              'n_estimators': [14, 15],
              'max_features': [0.4],
              'min_samples_split': [3, 4]}
clf = tree.DecisionTreeClassifier()
clfcv = GridSearchCV(estimator = clf, param_grid = param_grid,  cv = 4)

In [28]:
# running the random forest model
rfc = ensemble.RandomForestClassifier()

rfc_cv = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 4)

rfc_cv.fit(X_train, Y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['entropy', 'gini'], 'max_depth': [10],
                         'max_features': [0.4], 'min_samples_split': [3, 4],
                         'n_estimators': [14, 15]})

In [58]:
# get the f1 score and report
predict_test = rfc_cv.predict(X_test)
print('random forest is good?')
print(metrics.classification_report(predict_test, Y_test))

random forest is good?
              precision    recall  f1-score   support

           0       0.51      0.63      0.57      3029
           1       0.60      0.56      0.58      7763
           2       0.41      0.55      0.47      1265
           3       0.80      0.64      0.71      2738
           4       0.01      0.33      0.03         3
           5       0.00      0.00      0.00         6
           6       0.06      0.17      0.09        12
           7       0.70      0.58      0.64      1220

    accuracy                           0.59     16036
   macro avg       0.39      0.43      0.38     16036
weighted avg       0.61      0.59      0.59     16036



In [59]:
rfc_cv.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 0.4,
 'min_samples_split': 3,
 'n_estimators': 15}

In [61]:
# see the MSE
metrics.mean_squared_error(Y_test, predict_test)

2.6338862559241707

In [31]:
# now it comes to predict the test data
total_test = pd.read_csv('../input/2020125/use this test.csv')

In [32]:
# same procedure to remove outlier, and b means the threshold

def get_threshold_test(a):
    b = 0.98
    k = []
    m = 0
    if a in list(total_test.columns):
        order = list(total_test.columns).index(a)
        a = total_test.iloc[:,order].value_counts()
        a = pd.DataFrame(a).reset_index()
        
        for i in range(len(a)):
            if m < (len(total_test) * b):
                m += a.iloc[i,1]
            else:
                k.append(a.iloc[i,0])
    else:
        print('not in the columns!')
    return k

In [33]:
#Columns in which outliers need to be removed
lines_needed = ['hour_offset', 'ccs', 'total_mails_by_sender', 'sender_freq_total_period', 'sender_freq_prev_year',
     'sender_freq_prev_week', 'sender_freq_prev_month', 'sender_freq_prev_six_months', 'images', 'urls',
    'chars_in_subject','chars_in_body', 'image_mean_value', 'image_std_value', 'urls_mean_value', 'urls_std_value',
     'chars_in_subject_mean_value', 'chars_in_subject_std_value', 'chars_in_body_mean_value', 'chars_in_body_std_value',
     'img_url_ratio', 'body_subj_ratio']
description_test = total_test.describe()
for item in lines_needed:
    values_need_to_be_deleted = get_threshold_test(item)
    for i in range(len(total_test)):
        if (total_test.at[i, item] in values_need_to_be_deleted):
            total_test.at[i, item] = list(description[item])[1]
    print('finished the ' + str(lines_needed.index(item)))

total_test.describe()

finished the 0
finished the 1
finished the 2
finished the 3
finished the 4
finished the 5
finished the 6
finished the 7
finished the 8
finished the 9
finished the 10
finished the 11
finished the 12
finished the 13
finished the 14
finished the 15
finished the 16
finished the 17
finished the 18
finished the 19
finished the 20
finished the 21


,ID,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,...,data_type_multipart,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed
count,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,...,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000,34365.000000
mean,17182.000000,4.839923,0.294689,0.002590,1196.924196,2.629005,2.081995,4.147031,3.029435,2.428600,...,0.828081,0.162578,0.000029,0.727310,0.128416,0.075018,0.034163,0.025491,0.000058,0.000175
std,9920.465337,6.255755,0.837631,0.050825,2007.124761,3.704291,3.289611,7.829904,4.299556,3.600142,...,0.377316,0.368986,0.005394,0.445349,0.334557,0.263425,0.181649,0.157613,0.007629,0.013213
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8591.000000,0.000000,0.000000,0.000000,25.000000,0.076398,0.043836,0.285714,0.133333,0.065574,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17182.000000,2.000000,0.000000,0.000000,201.000000,0.383754,0.326027,0.857143,0.600000,0.408840,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,25773.000000,5.500000,0.000000,0.000000,1235.000000,5.979866,2.550685,6.000000,5.645161,3.704918,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,34364.000000,20.000000,5.000000,1.000000,8372.000000,21.000000,10.671233,68.571429,17.700000,12.087432,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
# delete the unnecessary columns
total_test = total_test.drop(['first_mail_of_sender'], axis = 1)
total_test = total_test.drop(['ID'], axis = 1)

In [35]:
total_test.to_csv('total_test_removed_outlier_with_0.98.csv')

In [36]:
# normalization as before

stdsc=StandardScaler()
normalized_total_test = stdsc.fit_transform(total_test)
normalized_total_test = pd.DataFrame(normalized_total_test)

In [37]:
normalized_total_test.head()

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,0.105517,-0.351818,-0.050957,3.009860,1.407257,1.606635,0.364374,0.555861,0.828991,-0.694861,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
1,2.263561,-0.351818,-0.050957,-0.570937,0.100154,-0.590434,-0.037024,0.914123,-0.596327,0.183824,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
2,0.105517,0.842042,-0.050957,0.608878,1.084326,1.381765,0.802262,1.838833,1.763111,-0.694861,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
3,-0.773687,-0.351818,-0.050957,-0.508159,-0.678842,-0.503817,-0.401930,-0.569553,-0.532690,0.936983,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
4,1.943851,-0.351818,-0.050957,-0.199753,-0.621370,-0.126534,0.364374,0.280009,0.203223,1.564615,...,-2.194696,2.269556,-0.005394,-1.633146,2.605229,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215


In [38]:
column_name = column_name[0:132]

In [39]:
column_name

Index(['hour_offset', 'ccs', 'bcced', 'total_mails_by_sender',
       'sender_freq_total_period', 'sender_freq_prev_year',
       'sender_freq_prev_week', 'sender_freq_prev_month',
       'sender_freq_prev_six_months', 'images',
       ...
       'data_type_multipart', 'data_type_text', 'source_type_IDM',
       'source_type_alternative', 'source_type_html', 'source_type_mixed',
       'source_type_plain', 'source_type_related', 'source_type_report',
       'source_type_signed'],
      dtype='object', length=132)

In [40]:
rename_dict = {}
for i in range(132):
    rename_dict.update({i: column_name[i]}, axis = 1)

In [41]:
normalized_total_test.rename(columns = rename_dict, inplace = True)

In [42]:
normalized_total_test.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_multipart,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed
0,0.105517,-0.351818,-0.050957,3.009860,1.407257,1.606635,0.364374,0.555861,0.828991,-0.694861,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
1,2.263561,-0.351818,-0.050957,-0.570937,0.100154,-0.590434,-0.037024,0.914123,-0.596327,0.183824,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
2,0.105517,0.842042,-0.050957,0.608878,1.084326,1.381765,0.802262,1.838833,1.763111,-0.694861,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
3,-0.773687,-0.351818,-0.050957,-0.508159,-0.678842,-0.503817,-0.401930,-0.569553,-0.532690,0.936983,...,0.455644,-0.440615,-0.005394,0.612315,-0.383843,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215
4,1.943851,-0.351818,-0.050957,-0.199753,-0.621370,-0.126534,0.364374,0.280009,0.203223,1.564615,...,-2.194696,2.269556,-0.005394,-1.633146,2.605229,-0.284785,-0.188072,-0.161734,-0.007629,-0.013215


In [43]:
total_test.head()

,hour_offset,ccs,bcced,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,images,...,data_type_multipart,data_type_text,source_type_IDM,source_type_alternative,source_type_html,source_type_mixed,source_type_plain,source_type_related,source_type_report,source_type_signed
0,5.5,0,0,7238,7.841820,7.367123,7.000000,5.419355,5.413043,0,...,1,0,0,1,0,0,0,0,0,0
1,19.0,0,0,51,3.000000,0.139726,3.857143,6.959703,0.281768,7,...,1,0,0,1,0,0,0,0,0,0
2,5.5,1,0,2419,6.645604,6.627397,10.428571,10.935484,8.775956,0,...,1,0,0,1,0,0,0,0,0,0
3,0.0,0,0,177,0.114415,0.424658,1.000000,0.580645,0.510870,13,...,1,0,0,1,0,0,0,0,0,0
4,17.0,0,0,796,0.327303,1.665753,7.000000,4.233333,3.160221,18,...,0,1,0,0,1,0,0,0,0,0


In [44]:
from collections import  Counter
predict_test = rfc_cv.predict(normalized_total_test)
#predict_test.value_count()
Counter(predict_test)

Counter({1: 20353, 0: 5847, 3: 3443, 7: 2373, 2: 2321, 6: 25, 5: 3})

In [45]:
predict_test = predict_test.astype(int)

In [46]:
predict_test

array([1, 0, 3, ..., 1, 1, 2])

In [47]:
for i in range(len(list(predict_test))):
    total_test.at[i, 'label'] = predict_test[i]
total_test = total_test['label']

In [48]:
total_test

0        1.0
1        0.0
2        3.0
3        1.0
4        0.0
        ... 
34360    7.0
34361    2.0
34362    1.0
34363    1.0
34364    2.0
Name: label, Length: 34365, dtype: float64

In [49]:
total_test = pd.DataFrame(total_test)
total_test

,label
0,1.0
1,0.0
2,3.0
3,1.0
4,0.0
...,...
34360,7.0
34361,2.0
34362,1.0
34363,1.0


In [50]:
total_test.to_csv('2020.12.5-4.csv')


In [ ]:
# 以下是KNN方法

In [ ]:
total_train = pd.read_csv('../input/2020125/use_this.csv')
total_test = pd.read_csv('../input/2020125/use this test.csv')

In [ ]:
total_train = total_train.reset_index()

total_train.head()
print(total_train.shape)

In [ ]:
total_train = total_train.iloc[:,2:135]

In [ ]:
from sklearn.preprocessing import minmax_scale
# 对所有自变量数据作标准化处理
predictors = total_train.columns[:-1]
X = minmax_scale(total_train[predictors])

In [ ]:
total_train.head()

In [ ]:
total_train.label

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, total_train.label, test_size = 0.25)

In [ ]:
# 设置待测试的不同k值
from sklearn import neighbors
K = np.arange(1,np.ceil(np.log2(total_train.shape[0])))
# 构建空的列表，用于存储平均MSE
mse = []
for k in K:
    # 使用10重交叉验证的方法，比对每一个k值下KNN模型的计算MSE
    cv_result = model_selection.cross_val_score(neighbors.KNeighborsRegressor(n_neighbors = int(k), weights = 'distance'), 
                                                X_train, y_train, cv = 10, scoring='neg_mean_squared_error')
    mse.append((-1*cv_result).mean())
    print(str(k) +' has finished')

In [ ]:
np.arange(1,np.ceil(np.log2(total_train.shape[0])))

In [ ]:
# 从k个平均MSE中挑选出最小值所对应的下标  

arg_min = np.array(mse).argmin()
# 绘制不同K值与平均MSE之间的折线图
plt.plot(K, mse)
# 添加点图
plt.scatter(K, mse)
# 添加文字说明
plt.text(K[arg_min], mse[arg_min] + 0.5, '最佳k值为%s' %int(K[arg_min]))
# 显示图形
plt.show()

In [ ]:
# 重新构建模型，并将最佳的近邻个数设置为7
knn_reg = neighbors.KNeighborsRegressor(n_neighbors = 16, weights = 'distance')
# 模型拟合
knn_reg.fit(X_train, y_train)
# 模型在测试集上的预测
predict = knn_reg.predict(X_test)
# 计算MSE值
metrics.mean_squared_error(y_test, predict)

In [ ]:
pd.DataFrame({'Real':y_test,'Predict':predict}, columns=['Real','Predict']).head(10)

In [ ]:
# 进行预测
total_test = pd.read_csv('../input/2020125/use this test.csv')

In [ ]:
def get_threshold_test(a):
    b = 0.98
    k = []
    m = 0
    if a in list(total_test.columns):
        order = list(total_test.columns).index(a)
        a = total_test.iloc[:,order].value_counts()
        a = pd.DataFrame(a).reset_index()
        
        for i in range(len(a)):
            if m < (len(total_test) * b):
                m += a.iloc[i,1]
            else:
                k.append(a.iloc[i,0])
    else:
        print('not in the columns!')
    return k
# 需要修改outlier的列：
lines_needed = ['hour_offset', 'ccs', 'total_mails_by_sender', 'sender_freq_total_period', 'sender_freq_prev_year',
     'sender_freq_prev_week', 'sender_freq_prev_month', 'sender_freq_prev_six_months', 'images', 'urls',
    'chars_in_subject','chars_in_body', 'image_mean_value', 'image_std_value', 'urls_mean_value', 'urls_std_value',
     'chars_in_subject_mean_value', 'chars_in_subject_std_value', 'chars_in_body_mean_value', 'chars_in_body_std_value',
     'img_url_ratio', 'body_subj_ratio']
description_test = total_test.describe()
for item in lines_needed:
    values_need_to_be_deleted = get_threshold_test(item)
    for i in range(len(total_test)):
        if (total_test.at[i, item] in values_need_to_be_deleted):
            total_test.at[i, item] = list(description[item])[1]
    print('finished the ' + str(lines_needed.index(item)))

total_test.describe()

In [ ]:
total_test = total_test.drop(['first_mail_of_sender'], axis = 1)
total_test = total_test.drop(['ID'], axis = 1)
stdsc=StandardScaler()
normalized_total_test = stdsc.fit_transform(total_test)
normalized_total_test = pd.DataFrame(normalized_total_test)
column_name = column_name[0:132]
rename_dict = {}
for i in range(132):
    rename_dict.update({i: column_name[i]}, axis = 1)
normalized_total_test.rename(columns = rename_dict, inplace = True)
from collections import  Counter

predict_test = knn_reg.predict(normalized_total_test)
#predict_test.value_count()
Counter(predict_test)

In [ ]:
predict_test = predict_test.astype(int)
for i in range(len(list(predict_test))):
    total_test.at[i, 'label'] = predict_test[i]
total_test = total_test['label']

In [ ]:
total_test = pd.DataFrame(total_test)
total_test

In [ ]:
total_test.to_csv('2020.12.6-666979.csv')